<a href="https://colab.research.google.com/github/manojmanivannan/ApacheSparkEssentials/blob/master/ML_2_SparkByPluralsight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Environment

In [1]:
!apt-get update
!apt-get install -y openjdk-8-jdk-headless scala jq  > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!pip install -q pyspark findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,567 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,095 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:1

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
!python --version

Python 3.8.15


In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# Spark session & context
conf = SparkConf().set('spark.ui.port', '4050')
try:
  sc = SparkContext(conf=conf)
except ValueError:
  sc.stop()
  sc = SparkContext(conf=conf)

spark = SparkSession.builder.master('local[2]').getOrCreate()

In [5]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [6]:
# create new token from ngrok
get_ipython().system_raw('./ngrok authtoken 2Icy8FQ3TBbFTdCf8HdmTNg2Gna_7HwR9ytuwftMCCyeNoS4n')
# revoke the token once you close the notebook
get_ipython().system_raw('./ngrok http 4050 &')

In [7]:
!sleep 10 && curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://cec8-34-148-74-81.ngrok.io


In [9]:
rawData = spark.read\
                .format('csv')\
                .option('header','true')\
                .load('titanic.csv')

In [10]:
rawData.toPandas().head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,None,S


In [11]:
from pyspark.sql.functions import col

dataset = rawData.select(col('Survived').cast('float'),
                         col('Pclass').cast('float'),
                         col('Sex'),
                         col('Age').cast('float'),
                         col('Fare').cast('float'),
                         col('Embarked'))

dataset.toPandas().head()

,Survived,Pclass,Sex,Age,Fare,Embarked
0,0.0,3.0,male,22.0,7.250000,S
1,1.0,1.0,female,38.0,71.283302,C
2,1.0,3.0,female,26.0,7.925000,S
3,1.0,1.0,female,35.0,53.099998,S
4,0.0,3.0,male,35.0,8.050000,S


In [12]:
dataset.count()

891

In [13]:
dataset = dataset.replace('?',None).dropna(how='any')
dataset.count()

712

In [14]:
from pyspark.ml.feature import StringIndexer

dataset = StringIndexer(
    inputCol='Sex',
    outputCol='Gender',
    handleInvalid='keep'
).fit(dataset).transform(dataset)

dataset = StringIndexer(
    inputCol='Embarked',
    outputCol='Boarded',
    handleInvalid='keep'
).fit(dataset).transform(dataset)

dataset.toPandas().head()


,Survived,Pclass,Sex,Age,Fare,Embarked,Gender,Boarded
0,0.0,3.0,male,22.0,7.250000,S,0.0,0.0
1,1.0,1.0,female,38.0,71.283302,C,1.0,1.0
2,1.0,3.0,female,26.0,7.925000,S,1.0,0.0
3,1.0,1.0,female,35.0,53.099998,S,1.0,0.0
4,0.0,3.0,male,35.0,8.050000,S,0.0,0.0


In [16]:
dataset = dataset.drop('Sex','Embarked')
dataset.toPandas().head()

,Survived,Pclass,Age,Fare,Gender,Boarded
0,0.0,3.0,22.0,7.250000,0.0,0.0
1,1.0,1.0,38.0,71.283302,1.0,1.0
2,1.0,3.0,26.0,7.925000,1.0,0.0
3,1.0,1.0,35.0,53.099998,1.0,0.0
4,0.0,3.0,35.0,8.050000,0.0,0.0


In [18]:
requiredFeatures = ['Survived', 'Pclass', 'Age', 'Fare', 'Gender', 'Boarded']

In [20]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=requiredFeatures, outputCol='features')

In [21]:
transformed_data = assembler.transform(dataset)

In [22]:
transformed_data.toPandas().head()

,Survived,Pclass,Age,Fare,Gender,Boarded,features
0,0.0,3.0,22.0,7.250000,0.0,0.0,"[0.0, 3.0, 22.0, 7.25, 0.0, 0.0]"
1,1.0,1.0,38.0,71.283302,1.0,1.0,"[1.0, 1.0, 38.0, 71.2833023071289, 1.0, 1.0]"
2,1.0,3.0,26.0,7.925000,1.0,0.0,"[1.0, 3.0, 26.0, 7.925000190734863, 1.0, 0.0]"
3,1.0,1.0,35.0,53.099998,1.0,0.0,"[1.0, 1.0, 35.0, 53.099998474121094, 1.0, 0.0]"
4,0.0,3.0,35.0,8.050000,0.0,0.0,"[0.0, 3.0, 35.0, 8.050000190734863, 0.0, 0.0]"


In [37]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=8, seed=3)
model = kmeans.fit(transformed_data)

In [38]:
clusteredData = model.transform(transformed_data)

In [39]:
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(clusteredData)
print('Silhouette with squared euclidean distance = ',silhouette)

Silhouette with squared euclidean distance =  0.6314387412353603


In [40]:
centers = model.clusterCenters()
print('Cluster centers:')
for center in centers:
  print(center)

Cluster centers:
[ 0.23913043  2.17391304 51.47282609 16.22164997  0.20652174  0.20652174]
[ 0.26557377  2.76065574 25.32459016  9.60222564  0.27213115  0.21967213]
[1.00000000e+00 1.00000000e+00 3.53333333e+01 5.12329224e+02
 3.33333333e-01 1.00000000e+00]
[  0.73333333   1.          30.33333333 239.99193726   0.73333333
   0.53333333]
[  0.73333333   1.          32.43066667 131.18388341   0.63333333
   0.5       ]
[ 0.68888889  1.16666667 36.03333333 69.30111321  0.48888889  0.38888889]
[ 0.53658537  2.63414634  7.28353659 26.93688921  0.48780488  0.23170732]
[ 0.45263158  1.93684211 34.43684211 28.13162209  0.44210526  0.21052632]


In [41]:
clusteredData.toPandas().head()

,Survived,Pclass,Age,Fare,Gender,Boarded,features,prediction
0,0.0,3.0,22.0,7.250000,0.0,0.0,"[0.0, 3.0, 22.0, 7.25, 0.0, 0.0]",1
1,1.0,1.0,38.0,71.283302,1.0,1.0,"[1.0, 1.0, 38.0, 71.2833023071289, 1.0, 1.0]",5
2,1.0,3.0,26.0,7.925000,1.0,0.0,"[1.0, 3.0, 26.0, 7.925000190734863, 1.0, 0.0]",1
3,1.0,1.0,35.0,53.099998,1.0,0.0,"[1.0, 1.0, 35.0, 53.099998474121094, 1.0, 0.0]",5
4,0.0,3.0,35.0,8.050000,0.0,0.0,"[0.0, 3.0, 35.0, 8.050000190734863, 0.0, 0.0]",1


In [42]:
from pyspark.sql.functions import *

dataset.select(avg('Survived'),
               avg('Pclass'),
               avg('Age'),
               avg('Fare'),
               avg('Gender'),
               avg('Boarded')).toPandas()

,avg(Survived),avg(Pclass),avg(Age),avg(Fare),avg(Gender),avg(Boarded)
0,0.404494,2.240169,29.642093,34.567251,0.363764,0.261236


In [43]:
clusteredData.groupBy('prediction').agg(
                                      avg('Survived'),
                                      avg('Pclass'),
                                      avg('Age'),
                                      avg('Fare'),
                                      avg('Gender'),
                                      avg('Boarded'),
                                      count('prediction')
                                    ).orderBy('prediction').toPandas()

,prediction,avg(Survived),avg(Pclass),avg(Age),avg(Fare),avg(Gender),avg(Boarded),count(prediction)
0,0,0.239130,2.173913,51.472826,16.221650,0.206522,0.206522,92
1,1,0.265574,2.760656,25.324590,9.602226,0.272131,0.219672,305
2,2,1.000000,1.000000,35.333333,512.329224,0.333333,1.000000,3
3,3,0.733333,1.000000,30.333333,239.991937,0.733333,0.533333,15
4,4,0.733333,1.000000,32.430667,131.183883,0.633333,0.500000,30
5,5,0.688889,1.166667,36.033333,69.301113,0.488889,0.388889,90
6,6,0.536585,2.634146,7.283537,26.936889,0.487805,0.231707,82
7,7,0.452632,1.936842,34.436842,28.131622,0.442105,0.210526,95


In [45]:
clusteredData.filter(clusteredData.prediction ==5).toPandas()

,Survived,Pclass,Age,Fare,Gender,Boarded,features,prediction
0,1.0,1.0,38.0,71.283302,1.0,1.0,"[1.0, 1.0, 38.0, 71.2833023071289, 1.0, 1.0]",5
1,1.0,1.0,35.0,53.099998,1.0,0.0,"[1.0, 1.0, 35.0, 53.099998474121094, 1.0, 0.0]",5
2,0.0,1.0,54.0,51.862499,0.0,0.0,"[0.0, 1.0, 54.0, 51.86249923706055, 0.0, 0.0]",5
3,0.0,1.0,28.0,82.170799,0.0,1.0,"[0.0, 1.0, 28.0, 82.1707992553711, 0.0, 1.0]",5
4,0.0,1.0,42.0,52.000000,0.0,0.0,"[0.0, 1.0, 42.0, 52.0, 0.0, 0.0]",5
...,...,...,...,...,...,...,...,...
85,1.0,1.0,39.0,83.158302,1.0,1.0,"[1.0, 1.0, 39.0, 83.1583023071289, 1.0, 1.0]",5
86,1.0,3.0,32.0,56.495800,0.0,0.0,"[1.0, 3.0, 32.0, 56.49580001831055, 0.0, 0.0]",5
87,0.0,1.0,31.0,50.495800,0.0,0.0,"[0.0, 1.0, 31.0, 50.49580001831055, 0.0, 0.0]",5
88,1.0,1.0,47.0,52.554199,1.0,0.0,"[1.0, 1.0, 47.0, 52.55419921875, 1.0, 0.0]",5
